In [1]:
# Week 2 — Turn the Model into a Decision System

# This notebook evaluates baseline vs AI routing decisions and logs
# results to a SQL database for auditability and analysis.

# I use DuckDB as a local analytical store for routing decisions.
# In production this would map to Postgres / a data warehouse.

# Block A : Decision Logging Backbone



In [2]:
import duckdb
from pathlib import Path

DB_PATH = Path("warehouse/routes.duckdb")
DB_PATH.parent.mkdir(exist_ok=True)

con = duckdb.connect(DB_PATH)
print("Connected to:", DB_PATH)


Connected to: warehouse/routes.duckdb


In [3]:
con.execute("""
CREATE TABLE IF NOT EXISTS runs (
    run_id TEXT PRIMARY KEY,
    ts TIMESTAMP,
    city TEXT,
    graph_version TEXT,
    model_version TEXT,
    notes TEXT
);
""")

con.execute("""
CREATE TABLE IF NOT EXISTS routes (
    run_id TEXT,
    route_type TEXT,        -- 'baseline' or 'ai'
    origin_node BIGINT,
    dest_node BIGINT,
    total_dist_m DOUBLE,
    total_time_s DOUBLE,
    n_edges INTEGER,
    PRIMARY KEY (run_id, route_type, origin_node, dest_node)
);
""")

con.execute("""
CREATE TABLE IF NOT EXISTS route_legs (
    run_id TEXT,
    route_type TEXT,
    origin_node BIGINT,
    dest_node BIGINT,
    leg_idx INTEGER,
    u BIGINT,
    v BIGINT,
    key INTEGER,
    length_m DOUBLE,
    tt_base_s DOUBLE,
    tt_pred_s DOUBLE
);
""")

print("Tables created.")


Tables created.


In [4]:
import joblib
import osmnx as ox
import networkx as nx
import pandas as pd
from datetime import datetime
import uuid

# Load graph (rebuild deterministically)
place = "Atlanta, Georgia, USA"
G = ox.graph_from_place(place, network_type="drive")

# Load ML model
model = joblib.load("artifacts/edge_tt_model.pkl")

edge_model = model["model"]
print(type(edge_model))





<class 'sklearn.ensemble._forest.RandomForestRegressor'>


In [5]:
# Picking ONE origin–destination pair
nodes = list(G.nodes)

origin = nodes[0]
destination = nodes[100]

print("Origin:", origin)
print("Destination:", destination)


Origin: 68145665
Destination: 68188935


In [6]:
# Computing Baseline route
baseline_path = nx.shortest_path(
    G,
    origin,
    destination,
    weight="length"
)

baseline_dist = 0

for u, v in zip(baseline_path[:-1], baseline_path[1:]):
    baseline_dist += G.edges[u, v, 0]["length"]

print("Baseline edges:", len(baseline_path))
print("Baseline distance (m):", round(baseline_dist, 2))


Baseline edges: 52
Baseline distance (m): 3841.46


In [7]:
# ------------------------------------------------------------
# Block B — AI Routing with Robust Edge Cost (MultiDiGraph-safe)
#
# Purpose:
# - Compute an AI-based route using predicted edge travel time
# - Safely handle MultiDiGraph edge attributes
# - Compute total predicted travel time for the route
#
# Assumptions:
# - G is a MultiDiGraph from OSMnx
# - origin and destination nodes already exist
# - Edge attributes may be:
#     - a single dict (with "length", "tt_pred_s")
#     - OR a dict of dicts (parallel edges)
# ------------------------------------------------------------

import networkx as nx

def edge_time(u, v, data):
    """
    Robust edge cost function for NetworkX shortest_path.

    NetworkX may pass:
    - a single edge-attribute dict
    - OR a dict of edge-attribute dicts (MultiDiGraph)

    We always select the first edge safely.
    """

    # If this is a dict of edges (MultiDiGraph case), grab first edge
    if "length" not in data:
        data = list(data.values())[0]

    # Use predicted travel time if available, else fallback
    return data.get("tt_pred_s", data["length"] / 10)


# ---- Compute AI route using predicted travel time ----
ai_path = nx.shortest_path(
    G,
    origin,
    destination,
    weight=edge_time
)

print("AI path computed. Nodes:", len(ai_path))


# ---- Compute total predicted travel time along the AI route ----
ai_time = 0.0

for u, v in zip(ai_path[:-1], ai_path[1:]):
    # Get edge data between u and v (MultiDiGraph-safe)
    edge_data = list(G.get_edge_data(u, v).values())[0]
    ai_time += edge_time(u, v, edge_data)

print("AI edges:", len(ai_path))
print("AI predicted total time (seconds):", round(ai_time, 2))


AI path computed. Nodes: 52
AI edges: 52
AI predicted total time (seconds): 384.15


In [19]:
# ------------------------------------------------------------
# Compute baseline total travel time (seconds)
# Using baseline edge travel time
# ------------------------------------------------------------

baseline_time = 0

for u, v in zip(baseline_path[:-1], baseline_path[1:]):
    data = list(G.get_edge_data(u, v).values())[0]

    # baseline time = length / baseline speed (fallback = 10 m/s)
    baseline_time += data.get("tt_baseline_s", data["length"] / 10)

print("Baseline total time (s):", round(baseline_time, 2))


Baseline total time (s): 384.15


In [20]:
# Week 2 — Block C: Evaluate Routing Decisions with SQL

# Goal:
# Evaluate baseline vs AI routing decisions using SQL over logged runs.

# This block answers:
# - Did AI improve routing?
# - By how much?
# - When did AI lose?
# - Where did time accumulate?


In [25]:
# ------------------------------------------------------------
# Week 2 — Block C (Step 1): Persist Routing Results to DuckDB
# ------------------------------------------------------------
# This cell takes routing results computed in Block B and
# writes them into SQL tables for auditing and analysis.
# ------------------------------------------------------------

import uuid
from datetime import datetime, timezone

# -----------------------
# Run metadata
# -----------------------
run_id = str(uuid.uuid4())
ts = datetime.now(timezone.utc)

# -----------------------
# Insert into RUNS table
# -----------------------
con.execute("""
INSERT INTO runs (
    run_id,
    ts,
    city,
    graph_version,
    model_version,
    notes
) VALUES (?, ?, ?, ?, ?, ?)
""", [
    run_id,
    ts,
    "Atlanta",
    "osm_atlanta_v1",
    "edge_tt_v1",
    "Baseline vs AI routing run"
])

# -----------------------
# Helper: log a route + its legs
# -----------------------
def log_route(route_type, path, total_time_s):
    """
    route_type: 'baseline' or 'ai'
    path: list of node IDs
    total_time_s: total travel time (seconds)
    """

    origin = path[0]
    dest = path[-1]
    n_edges = len(path) - 1

    # Compute total distance (meters)
    total_dist_m = 0
    for u, v in zip(path[:-1], path[1:]):
        edge_data = list(G.get_edge_data(u, v).values())[0]
        total_dist_m += edge_data.get("length", 0)

    # -----------------------
    # Insert into ROUTES table
    # -----------------------
    con.execute("""
    INSERT INTO routes (
        run_id,
        route_type,
        origin_node,
        dest_node,
        total_dist_m,
        total_time_s,
        n_edges
    ) VALUES (?, ?, ?, ?, ?, ?, ?)
    """, [
        run_id,
        route_type,
        origin,
        dest,
        total_dist_m,
        total_time_s,
        n_edges
    ])

    # -----------------------
    # Insert into ROUTE_LEGS table
    # -----------------------
    for i, (u, v) in enumerate(zip(path[:-1], path[1:])):
        # MultiDiGraph: take first edge safely
        key, data = list(G.get_edge_data(u, v).items())[0]

        con.execute("""
        INSERT INTO route_legs (
            run_id,
            route_type,
            origin_node,
            dest_node,
            leg_idx,
            u,
            v,
            key,
            length_m,
            tt_base_s,
            tt_pred_s
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, [
            run_id,
            route_type,
            origin,
            dest,
            i,
            u,
            v,
            key,
            data.get("length"),
            data.get("tt_baseline_s"),
            data.get("tt_pred_s")
        ])

# -----------------------
# Log baseline + AI routes
# -----------------------
log_route("baseline", baseline_path, baseline_time)
log_route("ai", ai_path, ai_time)

print("✅ Routing run logged successfully")
print("Run ID:", run_id)


✅ Routing run logged successfully
Run ID: aae86b7f-e68a-4fd4-b0a1-93f4ad547a44


In [26]:
con.execute("SELECT COUNT(*) AS n FROM runs").fetchdf()
con.execute("SELECT COUNT(*) AS n FROM routes").fetchdf()
con.execute("SELECT COUNT(*) AS n FROM route_legs").fetchdf()


,n
0,102


In [27]:
# ------------------------------------------------------------
# Query 1: Baseline vs AI total time comparison
# ------------------------------------------------------------

con.execute("""
SELECT
  b.run_id,
  b.total_time_s AS baseline_time_s,
  a.total_time_s AS ai_time_s,
  (b.total_time_s - a.total_time_s) AS time_saved_s,
  ROUND(
    (b.total_time_s - a.total_time_s) / b.total_time_s * 100, 2
  ) AS pct_time_saved
FROM routes b
JOIN routes a
  ON b.run_id = a.run_id
WHERE b.route_type = 'baseline'
  AND a.route_type = 'ai'
""").fetchdf()


,run_id,baseline_time_s,ai_time_s,time_saved_s,pct_time_saved
0,aae86b7f-e68a-4fd4-b0a1-93f4ad547a44,384.1458,384.1458,0.0,0.0


In [28]:
# ------------------------------------------------------------
# Query 2: Distance vs time tradeoff
# ------------------------------------------------------------

con.execute("""
SELECT
  route_type,
  total_dist_m,
  total_time_s
FROM routes
ORDER BY route_type
""").fetchdf()


,route_type,total_dist_m,total_time_s
0,ai,3841.458,384.1458
1,baseline,3841.458,384.1458


In [29]:
# ------------------------------------------------------------
# Query 3: Route complexity (number of edges)
# ------------------------------------------------------------

con.execute("""
SELECT
  route_type,
  n_edges
FROM routes
""").fetchdf()


,route_type,n_edges
0,baseline,51
1,ai,51


In [30]:
# ------------------------------------------------------------
# Query 4: Edge-level baseline vs AI time differences
# ------------------------------------------------------------

con.execute("""
SELECT
  u,
  v,
  length_m,
  tt_base_s,
  tt_pred_s,
  (tt_base_s - tt_pred_s) AS delta_s
FROM route_legs
WHERE route_type = 'ai'
ORDER BY delta_s DESC
LIMIT 15
""").fetchdf()


,u,v,length_m,tt_base_s,tt_pred_s,delta_s
0,68145665,462857057,74.605,NaN,NaN,NaN
1,462857057,462857016,68.456,NaN,NaN,NaN
2,462857016,68180812,154.375,NaN,NaN,NaN
3,68180812,68180808,160.853,NaN,NaN,NaN
4,68180808,68180804,128.333,NaN,NaN,NaN
5,68180804,68180790,111.330,NaN,NaN,NaN
6,68180790,68180784,43.962,NaN,NaN,NaN
7,68180784,68180776,76.577,NaN,NaN,NaN
8,68180776,68180759,193.081,NaN,NaN,NaN
9,68180759,68180754,164.690,NaN,NaN,NaN


In [31]:
# ------------------------------------------------------------
# Query 5: Where AI performs worse than baseline
# ------------------------------------------------------------

con.execute("""
SELECT
  u,
  v,
  length_m,
  tt_base_s,
  tt_pred_s,
  (tt_pred_s - tt_base_s) AS penalty_s
FROM route_legs
WHERE route_type = 'ai'
  AND tt_pred_s > tt_base_s
ORDER BY penalty_s DESC
LIMIT 10
""").fetchdf()


,u,v,length_m,tt_base_s,tt_pred_s,penalty_s


In [32]:
# ------------------------------------------------------------
# Query 6: Aggregate edge-level reliability metrics
# ------------------------------------------------------------

con.execute("""
SELECT
  route_type,
  COUNT(*) AS n_edges,
  AVG(tt_base_s) AS avg_baseline_s,
  AVG(tt_pred_s) AS avg_pred_s,
  AVG(tt_pred_s - tt_base_s) AS avg_delta_s
FROM route_legs
GROUP BY route_type
""").fetchdf()


,route_type,n_edges,avg_baseline_s,avg_pred_s,avg_delta_s
0,ai,51,NaN,NaN,NaN
1,baseline,51,NaN,NaN,NaN
